In [1]:
import pandas as pd

dataset = pd.read_csv('./dataset.csv', 
                    header=None, 
                    names=["Comment", "Movie" ,"Bool"])

dataset = dataset.tail(-1)
dataset = dataset.drop("Movie", 1)

dataset = dataset.sample(frac=.01, random_state=1).reset_index(drop=True)

dataset.head()

,Comment,Bool
0,\n ben bu kadar güzel bir...,"5,0"
1,"\n En yakın dostumun ""Eşş...","4,0"
2,\n son zamanlarda seyrett...,"0,5"
3,\n yer yer samimi ve amat...,"2,5"
4,\n Klasik korku içermeyen...,"2,0"


In [2]:
def map_points(x):
    x = float(x.replace(',', '.'))
    if x < 3.5:
        return False
    else:
        return True
def clean_values(x):
    x = x.replace('\n', ' ')
    x = x.replace('\W', ' ')
    x = x.lower()
    return x

dataset['Bool'] = dataset['Bool'].apply(map_points)
dataset['Comment'] = dataset['Comment'].apply(clean_values)
dataset['Comment'] = dataset['Comment'].str.split()

dataset.head()

,Comment,Bool
0,"[ben, bu, kadar, güzel, bir, film, ne, zamandı...",True
1,"[en, yakın, dostumun, ""eşşek, kadar, adam, old...",True
2,"[son, zamanlarda, seyrettiğim, en, kötü, film,...",False
3,"[yer, yer, samimi, ve, amatör, tadıyla, hafif,...",False
4,"[klasik, korku, içermeyen, kanlı, bir, film, b...",False


In [3]:
vocabulary = []
for comment in dataset['Comment']:
    for word in comment:
        vocabulary.append(word)
# zayıflıklardan biri datasetin temizlenmesine çok bağlı
vocabulary = list(set(vocabulary))

# datanın temizlenmesi ve sözlüğün oluşması burada bitiyor

print(vocabulary)

['oyuncluklar', 'bakarsanız', 'izleyin...', 'kahramanımızın', 'sydney', 'buzdolabında', 'şirkete', 'çalışmışlar.', 'dönüp', 'görüşünden,', 'kareografileriyle', 'isabetli', 'düşünün,ve', 'enteresan.ama', 'edilmelerine', 'boşlukta', 'alt', 'sanırsın', 'diil)', 'çıkmakta.', 'sunuyor.hollywood', 'deyildi', 'izliyoruz.', 'kurtaran', 'senaristi', 'this', "bernal'ın", 'vurmuş', 'manifestoda.', 'susalım', 'duygusu', 'soz', 'sırpı', 'ağlayabilirsiniz..', 'linklater)', 'emegi', 'otorite', 'sularında', 'ıcınden', 'hazırken', 'yanında,', "50'li", 'zekanın', 'sizlerin', 'vasat', 'çeker', '‘', 'bazıları', 'hayatımızda', 'gereken,mükemmel', 'alex', 'şekil', 'diyemem.reeves', 'yetersiz...', 'arasında', 'var.i̇nsanlarımız', 'nefeste', 'firari', 'oluyor.ayrıca', 'ekibinin', 'altında.aslında', 'konus', 'gerçekleştirmek', 'oyundan', 'su', 'ettiniz.', 'sevimliydi', 'kıyım,', 'durumdayım!', 'titanic', '10/9', 'gözüyle', 'anlar)..ve', 'pride', 'memleketimin', 'pitt', 'hazırlayacağım', 'donanımlı', 'yükseltmi

In [4]:
# frekans tablosu oluşturuyoruz
word_counts_per_comment = {unique_word: [0] * len(dataset['Comment']) for unique_word in vocabulary}

for index, comment in enumerate(dataset['Comment']):
    for word in comment:
        word_counts_per_comment[word][index] += 1

word_counts = pd.DataFrame(word_counts_per_comment)
word_counts.head()

,oyuncluklar,bakarsanız,izleyin...,kahramanımızın,sydney,buzdolabında,şirkete,çalışmışlar.,dönüp,"görüşünden,",...,etmiş,değilim.ankarada,kullanmış.,fikrim,hangi,kurtulmuş,bulanik,yapilmis,1-2,haksız
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
dataset_joined = pd.concat([dataset, word_counts], axis=1)

dataset_joined.head()

,Comment,Bool,oyuncluklar,bakarsanız,izleyin...,kahramanımızın,sydney,buzdolabında,şirkete,çalışmışlar.,...,etmiş,değilim.ankarada,kullanmış.,fikrim,hangi,kurtulmuş,bulanik,yapilmis,1-2,haksız
0,"[ben, bu, kadar, güzel, bir, film, ne, zamandı...",True,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"[en, yakın, dostumun, ""eşşek, kadar, adam, old...",True,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"[son, zamanlarda, seyrettiğim, en, kötü, film,...",False,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"[yer, yer, samimi, ve, amatör, tadıyla, hafif,...",False,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"[klasik, korku, içermeyen, kanlı, bir, film, b...",False,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
import numpy as np

positive_values = dataset_joined[dataset_joined['Bool'] == np.bool_(True)]
negative_values = dataset_joined[dataset_joined['Bool'] == np.bool_(False)]

#p(durum)
def find_class_prior_prob(positive_values, negative_values):
    positive_percentage = positive_values.shape[0] / len(dataset_joined)
    negative_percentage = negative_values.shape[0] / len(dataset_joined)
    
    return positive_percentage, negative_percentage
    
positive_percentage, negative_percentage = find_class_prior_prob(positive_values, negative_values)

#n değeri
def n(positive_values, negative_values):
    n_words_per_positive_message = positive_values['Comment'].apply(len)
    n_positive = n_words_per_positive_message.sum()

    n_words_per_negative_message = negative_values['Comment'].apply(len)
    n_negative = n_words_per_negative_message.sum()
    
    return n_positive, n_negative

n_positive, n_negative = nc(positive_values, negative_values)

#vocabulary uzunluğu
n_vocabulary = len(vocabulary)

alpha = 1 # laplace smoothing

In [7]:
parameters_positive = {unique_word:0 for unique_word in vocabulary}
parameters_negative = {unique_word:0 for unique_word in vocabulary}

# datasetteki her kelime için p(kelime|durum hesaplanması)

for word in vocabulary:
    #nc
    n_word_given_positive = positive_values[word].sum()
    #m estimate
    p_word_given_positive = (n_word_given_positive + alpha) / (n_positive + alpha*n_vocabulary)
    parameters_positive[word] = p_word_given_positive
    #nc
    n_word_given_negative = negative_values[word].sum()
    #m estimate
    p_word_given_negative = (n_word_given_negative + alpha) / (n_negative + alpha*n_vocabulary)
    parameters_negative[word] = p_word_given_negative

In [8]:
import re
import nltk
from stop_words import stop_words

WPT = nltk.WordPunctTokenizer()
# Yukarıdaki rowları normalize etme işlemine de bu method uygulanabilir
# bir ara implente et.
def norm_doc(single_doc):
    # TR: Dokümandan belirlenen özel karakterleri ve sayıları at
    # EN: Remove special characters and numbers
    single_doc = re.sub(" \d+", " ", single_doc)
    pattern = r"[{}]".format(",.;") 
    single_doc = re.sub(pattern, "", single_doc) 
    # TR: Dokümanı küçük harflere çevir
    # EN: Convert document to lowercase
    single_doc = single_doc.lower()
    single_doc = single_doc.strip()
    # TR: Dokümanı token'larına ayır
    # EN: Tokenize documents
    tokens = WPT.tokenize(single_doc)
    # TR: Stop-word listesindeki kelimeler hariç al
    # EN: Filter out the stop-words 
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # TR: Dokümanı tekrar oluştur
    # EN: Reconstruct the document
    single_doc = ' '.join(filtered_tokens)
    return single_doc

def norm_values(perc_positive, perc_negative):
    sum = perc_positive + perc_negative
    norm_positive = perc_positive / sum
    norm_negative = perc_negative / sum
    
    return norm_positive, norm_negative

def classify(message):
    message = re.sub('\W', ' ', message)
    message = norm_doc(message).split()

    p_positive_given_message = positive_percentage
    p_negative_given_message = negative_percentage

    for word in message:
        if word in parameters_positive:
            print("positive ", word, " ", parameters_positive[word])
            p_positive_given_message *= parameters_positive[word]
            
        if word in parameters_negative:
            print("negative ", word, " ", parameters_negative[word])
            p_negative_given_message *= parameters_negative[word]
    
    confidence = norm_values(p_positive_given_message, p_negative_given_message)

    if p_negative_given_message < p_positive_given_message:
        return True, confidence[0], confidence[1]
    elif p_negative_given_message > p_positive_given_message:
        return False, confidence[0], confidence[1]

In [9]:
classify("harika bir gün")

positive  harika   0.0010954576720250682
negative  harika   0.00011792916388222808
positive  gün   0.00030570911777443765
negative  gün   0.00019654860647038013


(True, 0.9614859353340232, 0.038514064665976844)